In [ ]:
# Importing libraries

import pandas as pd
import numpy as np
from math import *
import sys

# I am going to display only 7 columns to avoid problems of overlapping when publishing in web
pd.set_option('display.max_columns', 7)

# Importing visualization libraries
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import missingno as msno
plt.style.use( 'ggplot' )
%matplotlib inline

#Import preprocessing libraries
from sklearn.preprocessing import MinMaxScaler , StandardScaler, Imputer, LabelEncoder

# Ignore warnings 
import warnings
warnings.filterwarnings('ignore')
# Importing clustering libraries
from sklearn.cluster import KMeans


In [ ]:
# Reading the  data
df = pd.read_csv('C:\Users\Hector\Python_blog\CC_GENERAL.csv',index_col=0)


In [ ]:
# Here are the columns.
list(df.columns)
# All fields are the  typical that you may expect in a CC data set.
#---------------
#DATA DICTIONARY
#---------------

#Credit card holder ID 
#Monthly average balance (based on daily balance averages) 
#Ratio of last 12 months with balance 
#Total purchase amount spent during last 12 months 
#Total amount of one-off purchases 
#Total amount of installment purchases 
#Total cash-advance amount 
#Frequency of purchases (percentage of months with at least one purchase) 
#Frequency of one-off-purchases 
#Frequency of installment purchases 
#Cash-Advance frequency 
#Average amount per cash-advance transaction 
#Average amount per purchase transaction 
#Credit limit 
#Total payments(due amount paid by the customer to decrease their statement balance) in the period 
#Total minimum payments due in the period. 
#Percentage of months with full payment of the due statement balance 
#Number of months as a customer

In [ ]:
df.head()

In [ ]:
# We have 17 features in our dataframe
df.shape

In [ ]:
#we have 8950 rows and some nulls
df.info()


In [ ]:
df.describe()


In [ ]:
# Checking nulls
df.isnull().sum()
# We have nulls in credit limit and minimum payments
# I will imput the values later.

In [ ]:
# checking for nulls
# percentage of nan
# number of nan
msno.matrix(df);
msno.bar(df);

In [ ]:
# EXPORATORY DATA ANALYSIS
df.hist(figsize=(18,18));

In [ ]:
#let´s see how are distributed the frequency variables

df[['BALANCE_FREQUENCY',
 'PURCHASES_FREQUENCY',
 'ONEOFF_PURCHASES_FREQUENCY',
 'PURCHASES_INSTALLMENTS_FREQUENCY',
 'CASH_ADVANCE_FREQUENCY',
'PRC_FULL_PAYMENT']].plot.box(figsize=(18,10),title='Frequency',legend=True);
plt.tight_layout()
# We have data on Cash_advance_frequency that is wrong. I will clean the dataset later.
# There are also many outliers, but we will keep then for now

In [ ]:
#let´s see how are distributed the numeric variables

df[['BALANCE',
 'PURCHASES',
 'ONEOFF_PURCHASES',
 'INSTALLMENTS_PURCHASES',
 'CASH_ADVANCE',
 'CREDIT_LIMIT',
 'PAYMENTS',
 'MINIMUM_PAYMENTS'
]].plot.box(figsize=(18,10),title='Distribution',legend=True);
plt.tight_layout()

# There are also many outliers, but we will keep them for now

In [ ]:
#let´s see how are distributed the numeric variables

df[[ 'CASH_ADVANCE_TRX',
 'PURCHASES_TRX'
]].plot.box(figsize=(18,10),title='Distribution of transactions',legend=True);
plt.tight_layout()




# There are also many outliers, but we will keep them for now

In [ ]:
#let´s see how is distributed the tenure

df[['TENURE'
]].plot.kde(figsize=(18,10),title='Distribution of transactions',legend=True);
plt.tight_layout()

In [ ]:
#Lets take a look at how the  variables are correlated
plt.figure(figsize=(18,10))
sns.heatmap(df.corr(),cmap='coolwarm',annot=True);
plt.title('Correlations', size = 28);

In [ ]:
# Lets clean the data (inputing values and eliminating wrong data) before the segmentation
df.loc[(df['CASH_ADVANCE_FREQUENCY']>1)]
# we have 8 records for which the frequency is higher that 1. I will eliminate these  records

In [ ]:
# dropping the records with frequency higher that 1
df = df[(df[['CASH_ADVANCE_FREQUENCY']] <= 1).all(axis=1)]

In [ ]:
df.shape

In [ ]:
# Imputing values in 'MINIMUM_PAYMENTS' and 'CREDIT_LIMIT'
#I will use the median  to input the values
df['MINIMUM_PAYMENTS'].fillna(df['MINIMUM_PAYMENTS'].median(),inplace=True)
df['CREDIT_LIMIT'].fillna(df['CREDIT_LIMIT'].median(),inplace=True)

# I get rid of Customer Id as wee don´t need it
#df.reset_index(inplace=True)
#df.drop('CUST_ID',inplace=True,axis=1)

df.head()


In [ ]:
# Before using K-Means, as in K-means we optimize the sum of squared distances between the observations and their centroids
# and as some varibles are expresed in different variables i.e frequencies, currency amount and number of transactions,
# we need to standardize.
# I am going to leave this section commented, because later on I would like to explore the centroids, but standardizing 
# the data will give us better results. 
# Then, we would follow the analysis with dataframe df_scaled instead of df.

#Standardization
# Create the scaler object with a range of 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
df_columns=df.columns
# Fit on the data and transform
df_s=scaler.fit_transform(df.values)

# Create the new dataframe

df_scaled = pd.DataFrame(df_s,columns=df_columns)

#df.head()
df_scaled.head()

In [ ]:
# K MEANS  CLUSTERING
# Before using K means, i am going to determine the optimal number of clusters
# i will use the Elbow method to find a good number of clusters
inertia = []
for k in range(1, 20):
    kmeans = KMeans(n_clusters=k, random_state=1).fit(df)
    inertia.append(np.sqrt(kmeans.inertia_))

In [ ]:
plt.figure(figsize=(18,6))
plt.title('optimal number of clusters')
plt.plot(range(1, 20), inertia, marker='s');
plt.xlabel('$k$')
plt.ylabel('$J(C_k)$');

In [ ]:
# It seems that the optimal number of clusters is between 7.5  and 8.
# I am going to take 8 for the analysis
kmeans = KMeans(n_clusters=8)

In [ ]:
# applying kmeans
kmeans.fit(df)

In [ ]:
# Calculating the centroids

centroids=kmeans.cluster_centers_
centroids


In [ ]:
# Creating a dataframe for the centroids
centroids=kmeans.cluster_centers_
index=['C0','C1','C2','C3','C4','C5','C6','C7']
columns=df.columns
centroid_df=pd.DataFrame(centroids,index,columns)
centroid_df


In [ ]:
#Adding the clusters to the dataframe
df['cluster']=list(kmeans.labels_)

In [ ]:
df.head()

In [ ]:
sns.pairplot( df, hue="cluster")

In [ ]:
# I WILL REPEAT THE PROCESS BUT USING THE VARIABLES THAT HAVE MANAGED TO SEPARATE THE CLUSTERS MORE CLEARLY
# VISUALIZING,DESCRIBING AND EXPLAINING THE CLUSTERS

best_cols = ["BALANCE", "PURCHASES_FREQUENCY", "CASH_ADVANCE","INSTALLMENTS_PURCHASES",
             "CREDIT_LIMIT", "PAYMENTS","PRC_FULL_PAYMENT" ]

kmeans = KMeans(n_clusters=8, init="k-means++", n_init=10, max_iter=300) 
best_vals = df[best_cols].iloc[ :, :].values
y_pred = kmeans.fit_predict( best_vals )

df["cluster"] = y_pred
best_cols.append("cluster")
sns.pairplot( df[ best_cols ], hue="cluster");

In [ ]:
# Number of clients by cluster
df['cluster'].value_counts().plot.bar(figsize=(10,5), title='Customers by cluster');

df['cluster'].value_counts()

In [ ]:
# Creating a dataframe for the centroids
centroids=kmeans.cluster_centers_
index=['C0','C1','C2','C3','C4','C5','C6','C7']
columns=["BALANCE", "PURCHASES_FREQUENCY", "CASH_ADVANCE","INSTALLMENTS_PURCHASES",
             "CREDIT_LIMIT", "PAYMENTS","PRC_FULL_PAYMENT" ]
centroid_df=pd.DataFrame(centroids,index,columns)
centroid_df

In [ ]:
#CLUSTERS DESCRIPTION

#VH-very high, H- high, M- medium, L- low, VL- Very low
#C1    5105
#C3    1534
#C6     934
#C4     457
#C5     443
#C0     327
#C7      89
#C2      53

In [ ]:
#CLUSTER C0 -->327, VIP clients, strategy find ways for them to buy more. Mileage program
#----------
#BALANCE--> VH
#PURCHASE FREQUENCY-->M
#CASH ADVANCE-->H
#INSTALLMENTS PURCHASES-->H
#CREDIT LIMIT-->VH
#PAYMENTS-->M
#FULL PAYMENT-->L

In [ ]:
#let´s characterize the Persona in the Cluster CO
cluster_C0=df[df['cluster']==0]
cluster_C0[['BALANCE','CASH_ADVANCE','INSTALLMENTS_PURCHASES',
            'CREDIT_LIMIT','PAYMENTS']].plot.box(figsize=(18,10),by='cluster',title='Persona C0',legend=True);
plt.tight_layout()
cluster_C0[['PURCHASES_FREQUENCY','PRC_FULL_PAYMENT']].plot.box(figsize=(18,10),title='Persona C0',legend=True);
plt.tight_layout()

In [ ]:
#CLUSTER C1-->5105, bulk of the customers. Try for them to use the card more frequently. Rewards program.
#----------
#BALANCE--> L
#PURCHASE FREQUENCY-->M
#CASH ADVANCE-->L
#INSTALLMENTS PURCHASES-->L
#CREDIT LIMIT-->L
#PAYMENTS-->L
#FULL PAYMENT-->M

In [ ]:
#let´s characterize the Persona in the Cluster C1
cluster_C1=df[df['cluster']==1]
cluster_C1[['BALANCE','CASH_ADVANCE','INSTALLMENTS_PURCHASES',
            'CREDIT_LIMIT','PAYMENTS']].plot.box(figsize=(18,10),by='cluster',title='Persona C1',legend=True);
plt.tight_layout()
cluster_C1[['PURCHASES_FREQUENCY','PRC_FULL_PAYMENT']].plot.box(figsize=(18,10),title='Persona C1',legend=True);
plt.tight_layout()

In [ ]:
#CLUSTER C2-->53 VIP CUSTOMERS THAT BUY FREQUENTLY AND THE CARD TO WITHDROW MONEY. INCREASE THE CREDIT LIMIT AND OFFER T
#THEM LOANS
#----------
#BALANCE--> H
#PURCHASE FREQUENCY-->H
#CASH ADVANCE-->M
#INSTALLMENTS PURCHASES-->VH
#CREDIT LIMIT-->VH
#PAYMENTS-->VH
#FULL PAYMENT-->VH

In [ ]:
#let´s characterize the Persona in the Cluster C2
cluster_C2=df[df['cluster']==2]
cluster_C2[['BALANCE','CASH_ADVANCE','INSTALLMENTS_PURCHASES',
            'CREDIT_LIMIT','PAYMENTS']].plot.box(figsize=(18,10),by='cluster',title='Persona C2',legend=True);
plt.tight_layout()
cluster_C2[['PURCHASES_FREQUENCY','PRC_FULL_PAYMENT']].plot.box(figsize=(18,10),title='Persona C2',legend=True);
plt.tight_layout()

In [ ]:
#CLUSTER C3 -->1534, THESE ARE TRANSACTORS, NOT VERY PROFITABLE
#----------
#BALANCE--> L
#PURCHASE FREQUENCY-->H
#CASH ADVANCE-->L
#INSTALLMENTS PURCHASES-->M
#CREDIT LIMIT-->H
#PAYMENTS-->L
#FULL PAYMENT-->H


In [ ]:
#let´s characterize the Persona in the Cluster C3
cluster_C3=df[df['cluster']==3]
cluster_C3[['BALANCE','CASH_ADVANCE','INSTALLMENTS_PURCHASES',
            'CREDIT_LIMIT','PAYMENTS']].plot.box(figsize=(18,10),by='cluster',title='Persona C3',legend=True);
plt.tight_layout()
cluster_C3[['PURCHASES_FREQUENCY','PRC_FULL_PAYMENT']].plot.box(figsize=(18,10),title='Persona C3',legend=True);
plt.tight_layout()


In [ ]:
#CLUSTER C4 -->934, THESE CUSTOMERS USE THE CREDIT CARD FOR INSTALLMENT PURCHASES. tHEY ARE FINANCIALLY WISE.
#----------
#BALANCE--> M
#PURCHASE FREQUENCY-->H
#CASH ADVANCE-->L
#INSTALLMENTS PURCHASES-->H
#CREDIT LIMIT-->VH
#PAYMENTS-->M
#FULL PAYMENT-->H

In [ ]:
#let´s characterize the Persona in the Cluster C4
cluster_C4=df[df['cluster']==4]
cluster_C4[['BALANCE','CASH_ADVANCE','INSTALLMENTS_PURCHASES',
            'CREDIT_LIMIT','PAYMENTS']].plot.box(figsize=(18,10),by='cluster',title='Persona C4',legend=True);
plt.tight_layout()
cluster_C4[['PURCHASES_FREQUENCY','PRC_FULL_PAYMENT']].plot.box(figsize=(18,10),title='Persona C4',legend=True);
plt.tight_layout()

In [ ]:
#CLUSTER C5-->457. INCREASE THE CREDIT LIMIT OF THESE CLIENTS
#----------
#BALANCE--> M
#PURCHASE FREQUENCY-->M
#CASH ADVANCE-->M
#INSTALLMENTS PURCHASES-->H
#CREDIT LIMIT-->H
#PAYMENTS-->H
#FULL PAYMENT-->H

In [ ]:
#let´s characterize the Persona in the Cluster C5
cluster_C5=df[df['cluster']==5]
cluster_C5[['BALANCE','CASH_ADVANCE','INSTALLMENTS_PURCHASES',
            'CREDIT_LIMIT','PAYMENTS']].plot.box(figsize=(18,10),by='cluster',title='Persona C5',legend=True);
plt.tight_layout()
cluster_C5[['PURCHASES_FREQUENCY','PRC_FULL_PAYMENT']].plot.box(figsize=(18,10),title='Persona C5',legend=True);
plt.tight_layout()


In [ ]:
#CLUSTER C6-->443, VERY PROFITABLE CUSTOMERS. THEY USE THE CARD AS A REVOLVER.
#----------
#BALANCE--> H
#PURCHASE FREQUENCY-->L
#CASH ADVANCE-->M
#INSTALLMENTS PURCHASES-->L
#CREDIT LIMIT-->H
#PAYMENTS-->L
#FULL PAYMENT-->L

In [ ]:
#let´s characterize the Persona in the Cluster C6
cluster_C6=df[df['cluster']==2]
cluster_C6[['BALANCE','CASH_ADVANCE','INSTALLMENTS_PURCHASES',
            'CREDIT_LIMIT','PAYMENTS']].plot.box(figsize=(18,10),by='cluster',title='Persona C6',legend=True);
plt.tight_layout()
cluster_C6[['PURCHASES_FREQUENCY','PRC_FULL_PAYMENT']].plot.box(figsize=(18,10),title='Persona C6',legend=True);
plt.tight_layout()

In [ ]:
#CLUSTER C7-->89 tHIS GROUP IS VERY PROFITABLE AS THE DRAW MONEY WITH THE CREDIT CARD AND THEY FORGET TO PAYBACK IT
#----------
#BALANCE--> H
#PURCHASE FREQUENCY-->L
#CASH ADVANCE-->VH
#INSTALLMENTS PURCHASES-->M
#CREDIT LIMIT-->VH
#PAYMENTS-->H
#FULL PAYMENT-->L

In [ ]:
#let´s characterize the Persona in the Cluster C7
cluster_C7=df[df['cluster']==7]
cluster_C7[['BALANCE','CASH_ADVANCE','INSTALLMENTS_PURCHASES',
            'CREDIT_LIMIT','PAYMENTS']].plot.box(figsize=(18,10),by='cluster',title='Persona C7',legend=True);
plt.tight_layout()
cluster_C7[['PURCHASES_FREQUENCY','PRC_FULL_PAYMENT']].plot.box(figsize=(18,10),title='Persona C7',legend=True);
plt.tight_layout()

In [ ]:
#we can continue exploring the clusters in more detail
best_cols = ["CASH_ADVANCE","PAYMENTS","cluster"]

fig=sns.pairplot( df[ best_cols ], hue="cluster");
fig;
fig.savefig('clusters.png')

In [ ]:
#we can continue exploring the clusters in more detail
best_cols = ["BALANCE","PAYMENTS","cluster"]

fig=sns.pairplot( df[ best_cols ], hue="cluster");
fig;
fig.savefig('clusters.png')

In [ ]:
#we can continue exploring the clusters in more detail
best_cols = ["CREDIT_LIMIT","PAYMENTS","cluster"]

fig=sns.pairplot( df[ best_cols ], hue="cluster");
fig;
fig.savefig('clusters.png')

In [ ]:
#we can continue exploring the clusters in more detail
best_cols = ["CREDIT_LIMIT","INSTALLMENTS_PURCHASES","cluster"]

fig=sns.pairplot( df[ best_cols ], hue="cluster");
fig;
fig.savefig('clusters.png')

In [ ]:
#we can continue exploring the clusters in more detail
best_cols = ["CASH_ADVANCE","CREDIT_LIMIT","cluster"]

fig=sns.pairplot( df[ best_cols ], hue="cluster");
fig;
fig.savefig('clusters.png')